# Imports

In [1]:
import sys
sys.path.append('../../../')

from utils.load_finetuned_qlora_model import (
    load_tokenizer_and_finetuned_qlora_model_FALCON
)

import torch
# import bitsandbytes as bnb
import datetime
from peft import (
    PeftModel,
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)

from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer
)

import transformers
from datasets import load_dataset, Dataset 

import pandas as pd


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/matheusalb/anaconda3/envs/llm/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


------

# Functions

## Tokenize Prompts

In [2]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False,
        return_tensors=None
    )     
    return result

------

# Defining Parameters

In [3]:
model_name = 'tiiuae/falcon-7b'
SAVED_MODEL_PATH = './results/qlora_falcon7b_2k_trainer_2023-07-23T19:25:11.275576'
SAVED_TOKENIZER_PATH = './results/qlora_falcon7b_2k_trainer_2023-07-23T19:25:11.275576/tokenizer'
MAX_LENGTH=350

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

----

# Run

## Loading Model and Tokenizer

In [4]:
model, tokenizer = load_tokenizer_and_finetuned_qlora_model_FALCON(
    model_name,
    SAVED_MODEL_PATH,
    bnb_config,
    SAVED_TOKENIZER_PATH
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Inferencia

In [5]:
gen_input = lambda x: f'''\
<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
{x}
<assistant>:
'''.strip()

In [16]:
# generation_config=model.generation_config
# generation_config.use_cache = False
# generation_config.max_new_tokens=100
# generation_config['eos_token_id'] = tokenizer.eos_token_id
# generation_config['pad_token_id'] = tokenizer.pad_token_id
# generation_config.temeperature=0.7
# generation_config.top_k=0.9
# generation_config.num_return_sequences=1
# generation_config.pad_token_id=tokenizer.eos_token_id
# generation_config.eos_token_id=tokenizer.eos_token_id

def inference(model, text, temperature=0.01, top_p=1.0):
    enconded = tokenizer(text, return_tensors="pt")
    
    enconded.to('cuda:0')
    with torch.cuda.amp.autocast():
        outputs = model.generate(
            input_ids=enconded.input_ids,
            attention_mask=enconded.attention_mask,
            # generation_config=generation_config,
            temperature=temperature,
            top_p=top_p,
            max_new_tokens=350,
            use_cache=False,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=True,
        )
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [10]:
comment = 'Alimentação caríssima para um péssimo atendimento.. super mal atendido'
inference(model, gen_input(comment))

/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Alimentação caríssima para um péssimo atendimento.. super mal atendido
<assistant>: Caro cliente, lamentamos profundamente que sua experiência em nosso restaurante tenha sido insatisfatória devido ao atendimento e à qualidade da comida. Agradecemos por compartilhar sua opinião e gostaríamos de entender melhor os pontos negativos mencionados para melhorar nossos serviços.


In [14]:
#treino
comment='Filé estava duro. Ficamos decepcionados.'
inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Filé estava duro. Ficamos decepcionados.
<assistant>:


In [18]:
comment='''A falta de vaga para cadeirantes e idosos é uma falha terrível nos dias de hoje, além de ser lei não tivemos nenhum apoio ou mesmo interesse por parte dos funcionários para ajudar na questão do cadeirante que estava nos acompanhado.
Não vou deixar de avaliar os outros pontos, no que tange ao atendimento interno e as delícias que são servidas. tudo de bom. 
Mas não vá com cadeirante que passará sufoco.'''
inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
A falta de vaga para cadeirantes e idosos é uma falha terrível nos dias de hoje, além de ser lei não tivemos nenhum apoio ou mesmo interesse por parte dos funcionários para ajudar na questão do cadeirante que estava nos acompanhado.
Não vou deixar de avaliar os outros pontos, no que tange ao atendimento interno e as delícias que são servidas. tudo de bom. 
Mas não vá com cadeirante que passará sufoco.
<assistant>: Caro cliente, agradecemos por compartilhar sua experiência conosco. Lamentamos profundamente pela falta de vaga para cadeirantes e idosos em nosso estabelecimento. Reconhecemos que isso é uma falha terrível nos dias de hoje e estamos trabalhando para melhorar essa situação. Agradecemos seu feedback e vamos reforçar a importância de oferecer um atendimento adequado a todos os nossos clientes. Quanto à questão da cadeirante que estava acompanhando, pedimos descu

In [19]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
<assistant>: Olá, agradecemos pelo seu feedback. Lamentamos muito pela experiência negativa que você teve em nosso restaurante. Pedimos desculpas pelo atendimento demorado e pelo frango com gosto de peixe. Vamos investigar o ocorrido e tomar as medidas necessárias para melhorar esses pontos negativos. Agradecemos por nos informar sobre sua experiência e esperamos ter a oportunidade de recebê-la novamente para uma experiência melhor.


In [20]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
<assistant>: Olá, agradecemos pelo seu feedback. Lamentamos muito pela experiência negativa que você teve em nosso restaurante. Pedimos desculpas pelo atendimento demorado e pelo frango com gosto de peixe. Vamos investigar o ocorrido e tomar as medidas necessárias para melhorar esses pontos negativos. Agradecemos por nos informar sobre sua experiência e esperamos ter a oportunidade de recebê-la novamente para uma experiência melhor.


In [21]:
comment = 'Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.'
inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.
<assistant>: Caro cliente, agradecemos por compartilhar sua experiência conosco. Lamentamos que a temperatura do nosso estabelecimento tenha sido desconfortável e que a espera pela comida tenha sido longa. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Agradecemos seu feedback e esperamos ter a oportunidade de recebê-lo novamente em nosso restaurante.


In [22]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.
<assistant>: Caro cliente, agradecemos por compartilhar sua experiência conosco. Lamentamos profundamente pela demora no atendimento em nosso restaurante. Vamos reforçar a comunicação entre nossos funcionários para que isso não aconteça novamente. Agradecemos seu feedback e esperamos ter a oportunidade de recebê-lo novamente em nosso estabelecimento.


In [23]:
comment = 'O pior yakisoba que eu já comi na vida! Vem numa caixa com a massa toda amassada e o molho é horrível! Tive que jogar fora! Parecia comida de cachorro! Nunca mais!'
inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
O pior yakisoba que eu já comi na vida! Vem numa caixa com a massa toda amassada e o molho é horrível! Tive que jogar fora! Parecia comida de cachorro! Nunca mais!
<assistant>: Olá, agradecemos pelo seu feedback sobre o nosso yakisoba. Lamentamos muito que a sua experiência não tenha sido satisfatória. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Agradecemos por nos informar sobre essa questão.


In [24]:
comment= '''Cheguei para tomar café as 7h da manhã, pouquíssimas pessoas ,pedi um omelete levou 23 minutos a essa altura meu café já havia acabado, e chegou uma pessoa depois de mim e o pedido dela que era muito mais que o meu simples omelete saiu primeiro.
Péssimo atendimento .
Já fui bem atendida por outros  funcionários  mais essa turma da manhã 3 pessoas nota 2.'''
inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Cheguei para tomar café as 7h da manhã, pouquíssimas pessoas,pedi um omelete levou 23 minutos a essa altura meu café já havia acabado, e chegou uma pessoa depois de mim e o pedido dela que era muito mais que o meu simples omelete saiu primeiro.
Péssimo atendimento.
Já fui bem atendida por outros  funcionários  mais essa turma da manhã 3 pessoas nota 2.
<assistant>: Caro cliente, lamentamos profundamente pela experiência negativa que você teve em nosso restaurante. Pedimos desculpas pelo atraso na entrega do seu pedido e pela demora no atendimento. Reconhecemos que isso não correspondeu às nossas expectativas e que não atendeu às suas expectativas em relação ao atendimento. Levaremos suas críticas em consideração para melhorar nosso serviço. Agradecemos por compartilhar sua opinião e esperamos ter a oportunidade de recebê-lo novamente para uma experiência melhor.


In [25]:
#treino
comment='Não recomendo! Não faz muito meu estilo cantar para quem não conheço, coisa que é impossível sem marcar antes. Sim, existem vários ambientes tocando estilos diferentes mas é necessário muita sorte para conseguir um lugar onde todos gostem do que está tocando. Sem falar que, se tratando do karaokê: ninguém é profissional por isso as performances são horríveis e a musica e letra são retiradas do YouTube, mas mesmo assim o espaço insiste em cobrar entrada! Para finalizar a comida, que é bem cara, veio fria e não gostei de nada que comi! Nem a bebida salvou minha noite, paguei um absurdo pela menor porção de caipirinha que já vi! Teria feito melhor na minha casa!'

inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Não recomendo! Não faz muito meu estilo cantar para quem não conheço, coisa que é impossível sem marcar antes. Sim, existem vários ambientes tocando estilos diferentes mas é necessário muita sorte para conseguir um lugar onde todos gostem do que está tocando. Sem falar que, se tratando do karaokê: ninguém é profissional por isso as performances são horríveis e a musica e letra são retiradas do YouTube, mas mesmo assim o espaço insiste em cobrar entrada! Para finalizar a comida, que é bem cara, veio fria e não gostei de nada que comi! Nem a bebida salvou minha noite, paguei um absurdo pela menor porção de caipirinha que já vi! Teria feito melhor na minha casa!
<assistant>: Olá, agradecemos pelo seu feedback sobre a sua experiência em nosso restaurante. Lamentamos que a música e a letra tenham sido retiradas do YouTube, isso pode ter causado uma má impressão. Vamos revisa

In [26]:
#treino
comment = '''Eu gosto muito de tomar café aos domingos nessa panificadora, porém não sei o que esta acontecendo, ela está ficando muito suja.
Gosto da parte de cima,mas quando chego as mesas estão todas sujas, com xícaras, resto de lanches e etc.
Acredito que não estão higienizando.
Outro detalhe a tapioca está vindo muito salgada.
Espero que melhore, para que possamos continuar frequentando.'''
inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Eu gosto muito de tomar café aos domingos nessa panificadora, porém não sei o que esta acontecendo, ela está ficando muito suja.
Gosto da parte de cima,mas quando chego as mesas estão todas sujas, com xícaras, resto de lanches e etc.
Acredito que não estão higienizando.
Outro detalhe a tapioca está vindo muito salgada.
Espero que melhore, para que possamos continuar frequentando.
<assistant>: Olá, agradecemos pelo seu feedback sobre a nossa panificadora. Lamentamos muito pela sujeira e a falta de higienização. Vamos reforçar a importância da limpeza e higienização com nossa equipe. Quanto à tapioca, vamos revisar a preparação para garantir que esteja de acordo com as expectativas dos nossos clientes. Agradecemos por nos informar sobre esses pontos negativos e esperamos poder recebê-la novamente em breve.


In [27]:
comment = '''Eu gostaria de deixar minha indignação quanto o atendimento de um garçom.
Não estou  reclamando do restaurante.
Estive no restaurante umas duas vezes, foi tudo muito bom.
Mas me decepcionei na minha ultima vez.
Foi atendida por um garçom mal educado e sem ética pois o mesmo alem de ser Grosso ficava olhando para as minhas pernas. Fiquei muito constrangida com a situação.
Almoçamos tomamos. Sucos e cervejas, mesmo porque estavamos em 6 pessoas.
No final pedimos a conta e o mesmo  trouxe duas contas uma com mais itens incluindo a refeiçao, outra somente com as bebidas,achei estranho porque tinham me cobrado normal das últimas vezes. Em fim.
Como um gestor de um restaurante contrata pessoas sem conhecer seu carater?
Tudo bem que a mão de obra de estrangeiros seja mais barata. Mas vocês teriam que pensar no bem estar dos clientes.
Descobri o nome do garçom.
CARLOS AMED. Ele é cubano.
Ia fazer uma denuncia por assedio mas pensei na empresa. Por favor selecione melhor seus colaboradores.
Muito indignada.'''

inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Eu gostaria de deixar minha indignação quanto o atendimento de um garçom.
Não estou  reclamando do restaurante.
Estive no restaurante umas duas vezes, foi tudo muito bom.
Mas me decepcionei na minha ultima vez.
Foi atendida por um garçom mal educado e sem ética pois o mesmo alem de ser Grosso ficava olhando para as minhas pernas. Fiquei muito constrangida com a situação.
Almoçamos tomamos. Sucos e cervejas, mesmo porque estavamos em 6 pessoas.
No final pedimos a conta e o mesmo  trouxe duas contas uma com mais itens incluindo a refeiçao, outra somente com as bebidas,achei estranho porque tinham me cobrado normal das últimas vezes. Em fim.
Como um gestor de um restaurante contrata pessoas sem conhecer seu carater?
Tudo bem que a mão de obra de estrangeiros seja mais barata. Mas vocês teriam que pensar no bem estar dos clientes.
Descobri o nome do garçom.
CARLOS AMED. Ele

In [28]:
comment = "Fui no dia 10/12/21 às 11:00 o restaurante não tinha ninguém ainda, fui fazer um pedido para viagem. Depois de quase 30 minutos de espera percebo que tinham clientes que chegaram depois comendo...questionei um garçom pela demora do pedido foi averiguar e não deu nenhuma satisfação. Aí percebi depois que o pedido estava no balcão aguardando o favor de algum garçom pegar e olha que nem estava lotado o restaurante...desrespeito com o cliente e garçons mal treinados.. nunca mais nesse lugar."
inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Fui no dia 10/12/21 às 11:00 o restaurante não tinha ninguém ainda, fui fazer um pedido para viagem. Depois de quase 30 minutos de espera percebo que tinham clientes que chegaram depois comendo...questionei um garçom pela demora do pedido foi averiguar e não deu nenhuma satisfação. Aí percebi depois que o pedido estava no balcão aguardando o favor de algum garçom pegar e olha que nem estava lotado o restaurante...desrespeito com o cliente e garçons mal treinados.. nunca mais nesse lugar.
<assistant>: Caro cliente, lamentamos profundamente pela experiência negativa que você teve em nosso restaurante. Pedimos desculpas pela demora no atendimento e pela falta de atenção dos nossos garçons. Reconhecemos que isso causou uma má impressão e que não atendeu às suas expectativas. Levaremos suas observações em consideração para melhorar nossos serviços. Agradecemos por compartilh

In [29]:
comment = "Só não dou zero porque não dá e porque a torta de frango estava boazinha. Priorizam quem tem aparência de ter mais dinheiro e os garçons não fazem questão alguma de atender as pessoas. Olharam na minha cara, passaram direto até que eu implorasse pra ser atendida. :) Não anotaram meu pedido e ainda queriam colocar a culpa em mim pelo esquecimento. Enfim, não volto!"
inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Só não dou zero porque não dá e porque a torta de frango estava boazinha. Priorizam quem tem aparência de ter mais dinheiro e os garçons não fazem questão alguma de atender as pessoas. Olharam na minha cara, passaram direto até que eu implorasse pra ser atendida. :) Não anotaram meu pedido e ainda queriam colocar a culpa em mim pelo esquecimento. Enfim, não volto!
<assistant>: Caro cliente, lamentamos profundamente pela experiência negativa que você teve em nosso restaurante. Pedimos desculpas pelo atendimento insatisfatório que você recebeu e pela falta de atenção dos nossos garçons. Levaremos suas críticas em consideração para melhorar nossos serviços. Quanto à torta de frango, vamos revisar nossos processos para garantir que todos os pratos sejam preparados de forma adequada. Agradecemos seu feedback e esperamos ter a oportunidade de recebê-lo novamente para uma expe

In [30]:
comment = "Pedimos um prato de bife ao molho madeira sendo que o feijão veio com gosto de molho madeira e o bife veio com gosto de queijo então eu recomendaria que vocês se dedicassem mais e as atende vistas são muito de humor baixo"

inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Pedimos um prato de bife ao molho madeira sendo que o feijão veio com gosto de molho madeira e o bife veio com gosto de queijo então eu recomendaria que vocês se dedicassem mais e as atende vistas são muito de humor baixo
<assistant>: Olá, agradecemos pelo seu feedback. Lamentamos muito pela experiência negativa que você teve com o nosso prato de bife ao molho madeira. Vamos investigar o ocorrido e tomar as medidas necessárias para garantir que isso não se repita. Quanto ao feijão, vamos revisar nossos processos de preparo para garantir que não haja confusão de sabores. Quanto às atitudes de nossos atendentes, vamos reforçar a importância do bom humor e da cordialidade. Agradecemos por nos informar sobre esses pontos negativos e esperamos ter a oportunidade de recebê-la novamente para uma experiência melhor.


In [31]:
comment = "Pedi no ifood camarão empanado e mandaram camarão alho e óleo muito sem graça e poucas unidades. Fui inventar de experimentar por causa da propaganda no Instagram e me ferrei! Não recomendo."

inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Pedi no ifood camarão empanado e mandaram camarão alho e óleo muito sem graça e poucas unidades. Fui inventar de experimentar por causa da propaganda no Instagram e me ferrei! Não recomendo.
<assistant>: Olá, agradecemos pelo seu feedback sobre a sua experiência com o nosso camarão empanado. Lamentamos que a qualidade do seu pedido não tenha atendido às suas expectativas. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Agradecemos por nos informar e esperamos ter a oportunidade de melhor atendê-la no futuro.


In [32]:
comment = "A comida é boa, mas o atendimento é bem ruim e ainda fazem de tudo pra fazer com que o cliente esteja errado. Esperei 1:10h pelo meu pedido (picanha pra 3p) e depois de ver 2 mesas de pessoas que chegaram depois de mim, receberem seus pedido, reclamei com o garçom, que disse q ainda tinham 4 pedidos na minha frente. Quase enlouqueci. Mas na verdade, eles entregaram meu pedido na outra mesa e eles não queriam admitir. Só não fui embora pq estávamos em grupo e todos com fome. E ainda demorou mais 15min depois disso. Pedi desconto no jantar e não deram nem desculpas pelo ocorrido. NÃO recomendo o local por isso."
inference(model, gen_input(comment))

/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/transformers/generation/utils.py:1448: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
A comida é boa, mas o atendimento é bem ruim e ainda fazem de tudo pra fazer com que o cliente esteja errado. Esperei 1:10h pelo meu pedido (picanha pra 3p) e depois de ver 2 mesas de pessoas que chegaram depois de mim, receberem seus pedido, reclamei com o garçom, que disse q ainda tinham 4 pedidos na minha frente. Quase enlouqueci. Mas na verdade, eles entregaram meu pedido na outra mesa e eles não queriam admitir. Só não fui embora pq estávamos em grupo e todos com fome. E ainda demorou mais 15min depois disso. Pedi desconto no jantar e não deram nem desculpas pelo ocorrido. NÃO recomendo o local por isso.
<assistant>: Caro cliente, lamentamos profundamente pela experiência negativa que você teve em nosso restaurante. Pedimos desculpas pelo atraso no seu pedido e pela falta de comunicação adequada sobre o ocorrido. Reconhecemos que isso causou uma má impressão e que 

In [33]:
comment = "Salgados bem caros, sem tamanho ou sabor correspondente. Coxinhas por R$18 ou empanado por R$20. Compramos uma torta de R$200 e não nos permitiram comer a mesma na doceria? Tinha que pegar e ir comer em outro lugar pois é “política da empresa”. Lamentável, eu e minha família não retornamos mais, e somos consumidores fiéis de tortas por aí."
inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Salgados bem caros, sem tamanho ou sabor correspondente. Coxinhas por R$18 ou empanado por R$20. Compramos uma torta de R$200 e não nos permitiram comer a mesma na doceria? Tinha que pegar e ir comer em outro lugar pois é “política da empresa”. Lamentável, eu e minha família não retornamos mais, e somos consumidores fiéis de tortas por aí.
<assistant>: Caro cliente, agradecemos por compartilhar sua experiência conosco. Lamentamos que a quantidade e o tamanho dos salgados não tenham atendido às suas expectativas. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Quanto à política de não comer a torta em nosso estabelecimento, entendemos sua frustração e vamos reforçar a importância de respeitar essa regra. Agradecemos seu feedback e esperamos ter a oportunidade de recebê-lo novamente em nosso restaurante.


In [20]:
comment='''Lugar supostamente temático, com artistas de acordo com o tema piratas, porém, tocando axe, pagode e tudo que não coincide com o local.

Infelizmente é notório que o atendimento é enrolado, de baixa qualidade e muito demorado.

São cobrada taxas pelos usos de brinquedos e entrada com bolo de aniversário. Além disso tivemos problemas para unir mesas, como se fosse um crime fazer isso.

Não recomendo para pessoas que gostam de bom atendimento.'''

inference(model, gen_input(comment))

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Lugar supostamente temático, com artistas de acordo com o tema piratas, porém, tocando axe, pagode e tudo que não coincide com o local.

Infelizmente é notório que o atendimento é enrolado, de baixa qualidade e muito demorado.

São cobrada taxas pelos usos de brinquedos e entrada com bolo de aniversário. Além disso tivemos problemas para unir mesas, como se fosse um crime fazer isso.

Não recomendo para pessoas que gostam de bom atendimento.
<assistant>: Caro cliente, agradecemos por compartilhar sua experiência conosco. Lamentamos que a sua visita não tenha atendido às suas expectativas em relação ao tema piratas e ao tipo de música tocada. Vamos revisar nossos artistas para garantir que estejam mais em sintonia com o ambiente. Quanto ao atendimento, vamos reforçar a importância de um serviço de qualidade para todos os nossos clientes. Quanto às taxas pelos usos de bri

In [8]:
comment= '''
Num espaço maravilhoso, um Churrasco requentado, ressecado e caro. A aparência dos alimentos não foi convidativa. Pedimos  picanha, maminha, língua e carneiro. Pedimos vinagrete. A porção era pequena demais para 3 pessoas; e é feito com ingredientes tão baratos...
Insatisfação total. Oferecem bebida insistentemente na mesa. Jamais voltarei lá.
'''.strip()

inference(model, gen_input(comment))

/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Num espaço maravilhoso, um Churrasco requentado, ressecado e caro. A aparência dos alimentos não foi convidativa. Pedimos  picanha, maminha, língua e carneiro. Pedimos vinagrete. A porção era pequena demais para 3 pessoas; e é feito com ingredientes tão baratos...
Insatisfação total. Oferecem bebida insistentemente na mesa. Jamais voltarei lá.
<assistant>: Caro cliente, lamentamos profundamente que sua experiência em nosso restaurante não tenha sido satisfatória. Pedimos desculpas pela qualidade do churrasco, que não atendeu às suas expectativas. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Quanto à aparência dos alimentos, levaremos seu feedback em consideração para melhorar a apresentação. Quanto ao vinagrete, vamos revisar nossos ingredientes para garantir que ele esteja de acordo com as suas expectativas. Agradecemos por nos informar

In [17]:
comment= '''
Num espaço maravilhoso, um Churrasco requentado, ressecado e caro. A aparência dos alimentos não foi convidativa. Pedimos  picanha, maminha, língua e carneiro. Pedimos vinagrete. A porção era pequena demais para 3 pessoas; e é feito com ingredientes tão baratos...
Insatisfação total. Oferecem bebida insistentemente na mesa. Jamais voltarei lá.
'''.strip()

inference(model, gen_input(comment), temperature=0.2)

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Num espaço maravilhoso, um Churrasco requentado, ressecado e caro. A aparência dos alimentos não foi convidativa. Pedimos  picanha, maminha, língua e carneiro. Pedimos vinagrete. A porção era pequena demais para 3 pessoas; e é feito com ingredientes tão baratos...
Insatisfação total. Oferecem bebida insistentemente na mesa. Jamais voltarei lá.
<assistant>: Caro cliente,

Lamentamos profundamente que sua experiência em nosso restaurante não tenha sido satisfatória. Pedimos desculpas pela aparência e sabor insatisfatórios dos alimentos que foram servidos. Reconhecemos que a quantidade de comida oferecida não foi adequada para três pessoas e que a porção de vinagrete também foi insatisfatória.

Quanto à insistência em oferecer bebida, entendemos que isso possa ter sido desconfortável para você. Agradecemos por nos informar sobre essa questão e vamos reforçar a nossa equipe

In [18]:
comment= '''
Num espaço maravilhoso, um Churrasco requentado, ressecado e caro. A aparência dos alimentos não foi convidativa. Pedimos  picanha, maminha, língua e carneiro. Pedimos vinagrete. A porção era pequena demais para 3 pessoas; e é feito com ingredientes tão baratos...
Insatisfação total. Oferecem bebida insistentemente na mesa. Jamais voltarei lá.
'''.strip()

inference(model, gen_input(comment), temperature=0.2)

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Num espaço maravilhoso, um Churrasco requentado, ressecado e caro. A aparência dos alimentos não foi convidativa. Pedimos  picanha, maminha, língua e carneiro. Pedimos vinagrete. A porção era pequena demais para 3 pessoas; e é feito com ingredientes tão baratos...
Insatisfação total. Oferecem bebida insistentemente na mesa. Jamais voltarei lá.
<assistant>: Caro cliente, lamentamos muito que sua experiência em nosso restaurante não tenha sido satisfatória. Pedimos desculpas pela aparência e sabor dos alimentos, assim como pela quantidade inadequada do vinagrete. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Agradecemos seu feedback e esperamos ter a oportunidade de melhor atendê-lo no futuro.


In [23]:
comment= '''
Num espaço maravilhoso, um Churrasco requentado, ressecado e caro. A aparência dos alimentos não foi convidativa. Pedimos  picanha, maminha, língua e carneiro. Pedimos vinagrete. A porção era pequena demais para 3 pessoas; e é feito com ingredientes tão baratos...
Insatisfação total. Oferecem bebida insistentemente na mesa. Jamais voltarei lá.
'''.strip()

inference(model, gen_input(comment), temperature=0.01)

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Num espaço maravilhoso, um Churrasco requentado, ressecado e caro. A aparência dos alimentos não foi convidativa. Pedimos  picanha, maminha, língua e carneiro. Pedimos vinagrete. A porção era pequena demais para 3 pessoas; e é feito com ingredientes tão baratos...
Insatisfação total. Oferecem bebida insistentemente na mesa. Jamais voltarei lá.
<assistant>: Caro cliente, lamentamos profundamente que sua experiência em nosso restaurante não tenha sido satisfatória. Pedimos desculpas pela qualidade do churrasco, que não atendeu às suas expectativas. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Quanto à aparência dos alimentos, levaremos seu feedback em consideração para melhorar a apresentação. Quanto ao vinagrete, vamos revisar nossos ingredientes para garantir que seja mais satisfatório. Agradecemos por nos informar sobre a quantidade ins

In [32]:
comment= '''
Num espaço maravilhoso, um Churrasco requentado, ressecado e caro. A aparência dos alimentos não foi convidativa. Pedimos  picanha, maminha, língua e carneiro. Pedimos vinagrete. A porção era pequena demais para 3 pessoas; e é feito com ingredientes tão baratos...
Insatisfação total. Oferecem bebida insistentemente na mesa. Jamais voltarei lá.
'''.strip()

inference(model, gen_input(comment), temperature=0.2)

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Num espaço maravilhoso, um Churrasco requentado, ressecado e caro. A aparência dos alimentos não foi convidativa. Pedimos  picanha, maminha, língua e carneiro. Pedimos vinagrete. A porção era pequena demais para 3 pessoas; e é feito com ingredientes tão baratos...
Insatisfação total. Oferecem bebida insistentemente na mesa. Jamais voltarei lá.
<assistant>: Caro cliente, lamentamos profundamente que sua experiência em nosso restaurante não tenha sido satisfatória. Pedimos desculpas pela aparência e sabor insatisfatórios dos pratos que foram servidos. Vamos revisar nossos processos de preparo para garantir que isso não aconteça novamente. Quanto à quantidade dos pratos, vamos avaliar a quantidade adequada para servir em uma mesa. Agradecemos seu feedback e esperamos ter a oportunidade de melhor atendê-lo no futuro.


In [31]:
comment = '''
Comida cara e muito ruim. Comi picanha e estava muito dura, um arroz cremoso que estava totalmente sem tempero, muito ruim mesmo... O dadinho de tapioca foi frito em óleo Velho, pior que coxinha de péssima qualidade... O queijinho de entrada estava bom, as bebidas tbb... A conta deu 190,00, um casal com 1 criança. Pode ter certeza, que não voltaremos mais.
'''.strip()

inference(model, gen_input(comment), temperature=0.5)

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Comida cara e muito ruim. Comi picanha e estava muito dura, um arroz cremoso que estava totalmente sem tempero, muito ruim mesmo... O dadinho de tapioca foi frito em óleo Velho, pior que coxinha de péssima qualidade... O queijinho de entrada estava bom, as bebidas tbb... A conta deu 190,00, um casal com 1 criança. Pode ter certeza, que não voltaremos mais.
<assistant>: Olá,

Lamentamos muito pela experiência negativa que você teve em nosso restaurante. Pedimos desculpas pela qualidade da picanha e do arroz cremoso que você recebeu. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Quanto ao dadinho de tapioca, vamos verificar a qualidade do óleo que utilizamos para preparar nossos pratos.

Quanto à bebida, estamos felizes em saber que gostou. Agradecemos seu feedback sobre a conta também, vamos avaliar nossos preços para garantir que estejam 

In [37]:
comment = '''
Comida cara e muito ruim. Comi picanha e estava muito dura, um arroz cremoso que estava totalmente sem tempero, muito ruim mesmo... O dadinho de tapioca foi frito em óleo Velho, pior que coxinha de péssima qualidade... O queijinho de entrada estava bom, as bebidas tbb... A conta deu 190,00, um casal com 1 criança. Pode ter certeza, que não voltaremos mais.
'''.strip()

inference(model, gen_input(comment), temperature=0.5)

<human>: Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
Comida cara e muito ruim. Comi picanha e estava muito dura, um arroz cremoso que estava totalmente sem tempero, muito ruim mesmo... O dadinho de tapioca foi frito em óleo Velho, pior que coxinha de péssima qualidade... O queijinho de entrada estava bom, as bebidas tbb... A conta deu 190,00, um casal com 1 criança. Pode ter certeza, que não voltaremos mais.
<assistant>: Caro cliente, lamentamos muito pela experiência negativa que você teve em nosso restaurante. Pedimos desculpas pela comida que não atendeu às suas expectativas em relação ao preço e qualidade. Vamos revisar nossos processos de preparo para garantir que isso não aconteça novamente. Quanto à qualidade do arroz cremoso, vamos verificar o que aconteceu para que isso não se repita. Quanto ao dadinho de tapioca, vamos verificar a qualidade do óleo usado para preparar nossos pratos. Agradecemos por nos informar 